In [16]:
%pip install --upgrade minerl --user

Note: you may need to restart the kernel to use updated packages.


# Installing minerl
MineRL requires some dependencies to be installed. Notably specific JDKs. [Link to docs](https://minerl.io/docs/tutorials/index.html)

In [29]:
import minerl
# minerl.data.download('./data/')
# Set the environment variable MINERL_DATA_ROOT to the path of the data directory.
import os
os.environ['MINERL_DATA_ROOT'] = os.path.join(os.getcwd(), 'data')
print(os.environ['MINERL_DATA_ROOT'])
# Run: `python3 -m minerl.data.download --environment "MineRLTreechop-v0"`


/mnt/l/PythonRL/data


In [26]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


#Setting up PyTorch for GPU

In [ ]:
import torch
torch.cuda.is_available()
